In [69]:
from pathlib import Path
from datetime import datetime
from dateutil import tz

import numpy as np
from datetime import datetime
import xarray as xr
from xarray.backends import NetCDF4DataStore
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.animation import ArtistAnimation
from cartopy import crs as ccrs
from metpy.plots import USCOUNTIES
import metpy.calc as mpcalc
from scipy.ndimage import gaussian_filter

## Model Reflectivity

In [23]:
%%capture

# Define area to plot
BOTTOM_LAT = 23
TOP_LAT = 40
LEFT_LON = -97
RIGHT_LON = -70

# Model Endpoints 
hrrr = 'https://nomads.ncep.noaa.gov/dods/hrrr/hrrr20200726/hrrr_sfc.t18z'
arw = 'https://nomads.ncep.noaa.gov/dods/hiresw/hiresw20200430/hiresw_conusarw_12z'
nbm = 'https://nomads.ncep.noaa.gov/dods/blend/blend20200417/blend_1hr_22z'
rap = 'https://nomads.ncep.noaa.gov/dods/rap/rap20200503/rap_21z'
gfs = 'https://nomads.ncep.noaa.gov/dods/gfs_0p25/gfs20200429/gfs_0p25_18z'

base_url = hrrr

ds = xr.open_dataset(base_url, decode_times=True)
ds = ds.metpy.parse_cf()

# Take lat and lon slices
LAT_SLICE = slice(BOTTOM_LAT, TOP_LAT)

if base_url == gfs:
    LON_SLICE = slice(360 + LEFT_LON, 360 + RIGHT_LON)
else:
    LON_SLICE = slice(LEFT_LON, RIGHT_LON)
    
# Extract all lats, lons, and meteorological datasets to plot
lats = ds.lat.sel(lat=LAT_SLICE).values

if base_url == gfs:
    lons = ds.lon.sel(lon=LON_SLICE).values
else:
    lons = 360 + ds.lon.sel(lon=LON_SLICE).values

# Capture model variable we want to plot
data = ds.refcclm

## Time Conversion from UTC to Eastern Time

In [24]:
def convert_datetime_from_nomads_to_local(string):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('America/New_York')
    utc_time = datetime.strptime(string, '%Y-%m-%dT%H:%M:%S').replace(tzinfo=from_zone)
    eastern = utc_time.astimezone(to_zone)
    date_time = datetime.strftime(eastern, '%a, %b %d, %Y %I:%M %p').lstrip('0').replace(' 0', ' ')
    return date_time

## Setup Basemap
Build this outside of the forecast loop since these features stay the same

In [37]:
%%capture
# Setup matplotlib figure
fig = plt.figure(figsize=(1920/72, 1080/72))
ax = fig.add_axes([0, 0, 1, 1], projection=ccrs.PlateCarree())

 # Add state lines, coastlines, map extent
ax.add_feature(cfeature.OCEAN.with_scale('10m'), color='lightgray')
ax.add_feature(cfeature.COASTLINE.with_scale('10m'), linewidth=0.75)
ax.add_feature(USCOUNTIES.with_scale('20m'), edgecolor='lightgray', zorder=1, linewidth=0.75)
ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=1.00)
ax.set_extent([-89, -75, 23, 37])
ax.set_adjustable('datalim')

# Build colortable based on t=1 data
refl = data.isel(time=1)
refl = refl.metpy.sel(lat=LAT_SLICE, lon=LON_SLICE).squeeze().metpy.unit_array
reflectivity_values = np.arange(20, 80, 5)
contour_fill = ax.contourf(lons, lats, refl, reflectivity_values, alpha=0,
                           cmap=plt.cm.gist_ncar, transform=ccrs.PlateCarree())

cbar = plt.colorbar(contour_fill, orientation='horizontal', pad=0, aspect=50)
cbar.ax.tick_params(labelsize=20)
cbar.solids.set(alpha=1)

## Model Time Series
Function to plot model data and manage text/contours so they're removed before the next timestep is plotted

In [91]:
artists = []

for time_step in range(1, 4):
    # [contourf.remove() for contourf in contour_fill.collections if time_step > 1]
    
    refl = data.isel(time=time_step)
    local_time = convert_datetime_from_nomads_to_local(str(refl.time.values).split('.')[0])
    refl = refl.metpy.sel(lat=LAT_SLICE, lon=LON_SLICE).squeeze().metpy.unit_array

    reflectivity_values = np.arange(20, 80, 5)

    contour_fill = ax.contourf(lons, lats, refl, reflectivity_values, zorder=5,
                               cmap=plt.cm.gist_ncar, transform=ccrs.PlateCarree())
    
    add_arts = contour_fill.collections

    text = ax.text(0.01, 0.99, s=f'Model Simulated Radar\n{local_time}',
                   fontsize=32, fontweight='bold', fontname='Arial',
                   color='white', bbox={'facecolor': 'navy', 'alpha': 1.0, 'edgecolor': 'none'},
                   transform=ax.transAxes, zorder=10, va='top')
    
    artists.append(add_arts + [text])


## Display JS Animator within Jupyter Lab
Execute the code that builds the animation

In [92]:
plt.rcParams['animation.html'] = 'jshtml'


anim = ArtistAnimation(fig, artists, interval=200, blit=False, repeat_delay=1000)
anim

## Write MP4 and GIF

In [8]:
graphics_dir = '../..' / Path('.') / 'graphics' / 'models'

anim.save(f'{graphics_dir}/reflectivity_hrrr.mp4', writer='ffmpeg', fps=3)
anim.save(f'{graphics_dir}/reflectivity_hrrr.gif', writer='imagemagick', fps=2)